# Axelrod's Model of Social Norms and Meta-Norms

### Nathan Brei

### Technical University of Munich

We are going to start by defining our agent. In this particular simulation, the agent's behavior is going to be determined by two meme values, `boldness` and `viciousness`, as per Axelrod's paper [Axelrod, 1986]. Every time the agent interacts with its neighbors, their respective `payoff`s will be updated. These interactions are mediated via `isBad` and `visibility`, which describe whether the agent ultimately chose to defect and the probability that a  neighboring agent recognized it. After some number of interactions, the expected value of each agent's payoff will hopefully converge, and the agent can adjust its strategy (in other words, update boldness and viciousness) by looking at the strategies and payoffs of its neighbors.

Unlike Axelrod's original paper, this particular simulation takes spatial proximity into account. The agents 'live' on an abstract 2D grid and only interact with their immediate neighbors, like cellular automata.


In [1]:
type Agent
    
    boldness :: Float16
    viciousness :: Float16

    payoff :: Float16
    visibility :: Float16

    isBad :: Bool
    isMerciful :: Bool
    
    location :: Tuple{Int,Int}
end

We define a Landscape, which represents our 2D grid. We implement it as a plain 2D array, and use a type alias to avoid any unnecessary indirection. We make the following design decisions:

* The grid wraps around, giving us a torus topology, or periodic boundary conditions

* Every cell is occupied by exactly one agent. Note: we had other options:
    - Zero or one agents per cell:  `Nullable{Array{Agent, 2}}`
    - Zero or more agents per cell: `Array{Agent, 3}`
    
* Agents are immobile


The relationship between Agent and Landscape follows the array-of-structures pattern which is good for conceptual clarity. It is not so good for memory performance, so you could consider changing this to a structure-of-arrays. Remember that in Julia, a composite type such as Agent is a struct, not an object, and hence could be stored in-place. I have not yet verified that this is in fact the case. 

In [2]:
Landscape = Array{Agent, 2}

Array{Agent,2}

Now we want to create a new landscape filled with randomly generated agents. 
Although both Axelrod and Gaylord ch 5 ex 9 say we should represent our meme values as ints from 1..7, 
I'm using floats because I find it to be conceptually cleaner. 

In [3]:
function initialize_landscape(width::Int, height::Int)
    landscape = Landscape(height, width)
    for i = 1:height
        for j = 1:width
            landscape[i,j] = Agent(rand(), rand(), 0.0, 0.0, false, false, (i,j))
        end
    end
    return landscape :: Landscape
end

l = initialize_landscape(10, 10)

10×10 Array{Agent,2}:
 Agent(0.079346, 0.26758, 0.0, 0.0, false, false, (1, 1))   …  Agent(0.25684, 0.23438, 0.0, 0.0, false, false, (1, 10)) 
 Agent(0.57666, 0.11774, 0.0, 0.0, false, false, (2, 1))       Agent(0.83008, 0.58398, 0.0, 0.0, false, false, (2, 10)) 
 Agent(0.75488, 0.01458, 0.0, 0.0, false, false, (3, 1))       Agent(0.54443, 0.37793, 0.0, 0.0, false, false, (3, 10)) 
 Agent(0.084839, 0.13379, 0.0, 0.0, false, false, (4, 1))      Agent(0.074585, 0.66016, 0.0, 0.0, false, false, (4, 10))
 Agent(0.20923, 0.36182, 0.0, 0.0, false, false, (5, 1))       Agent(0.15576, 0.77832, 0.0, 0.0, false, false, (5, 10)) 
 Agent(0.63721, 0.28784, 0.0, 0.0, false, false, (6, 1))    …  Agent(0.33228, 0.23401, 0.0, 0.0, false, false, (6, 10)) 
 Agent(0.0084686, 0.67236, 0.0, 0.0, false, false, (7, 1))     Agent(0.63477, 0.2522, 0.0, 0.0, false, false, (7, 10))  
 Agent(0.55176, 0.49878, 0.0, 0.0, false, false, (8, 1))       Agent(0.076721, 0.98828, 0.0, 0.0, false, false, (8, 10))
 Agent(0.5

We define a function which takes an Agent and returns a list of neighboring Agents. This is where we enforce the torus topology constraint. 

Since our agents can not move, this is the primary coupling to the Landscape implementation. We should have our transition functions call `neighbors()` and pretend that the Landscape data structure is abstract and opaque. This way, we can swap out different Landscape implementations with minimal code changes. (In a true object-oriented language we'd use encapsulation for this.) 

The compiler will heavily optimize the list comprehensions, I hope. 

In [3]:
function neighbors(agent::Agent, landscape::Landscape)
    r,c = agent.location
    r_max, c_max = size(landscape)
    offsets = [(dx,dy) for dx = -1:1, dy = -1:1 if !(dx==0 && dy==0)]
    coords = [((r+dy+r_max-1)%r_max + 1, (c+dx+c_max-1)%c_max + 1) for (dx,dy) = offsets]
    agents = [landscape[r,c] for (r,c) = coords]
    return agents :: Array{Agent,1}
end

neighbors (generic function with 1 method)

Let's quickly verify that `neighbors()` correctly wraps around. The [1,1] agent should be neighbors with blocks of agents in each corner.

In [71]:
[a.location for a = neighbors(l[1,1],l)]

8-element Array{Tuple{Int64,Int64},1}:
 (10, 10)
 (10, 1) 
 (10, 2) 
 (1, 10) 
 (1, 2)  
 (2, 10) 
 (2, 1)  
 (2, 2)  

We have created a large landscape but as of yet have no way to visualize it. Because our landscape implementation is so dead simple, we can generate a heatmap showing each of (boldness, viciousness, payoff) over the domain.

In [53]:
using Plots
boldnesses = map((agent) -> agent.boldness, l)
heatmap(boldnesses, title="Agent boldness, generation 0")

We model interactions between agents via the transition function `update_payoff`, which runs for each agent in two separate phases, `defect` and `punish`. The two separate phases are necessary because `punish` looks for defections among all neighbors; doing this in a single pass would pick up spurious defection data from the previous iteration.

A third phase, `metapunish`, is optional. This represents the more complicated case of a population enforcing its norms by also punishing anyone who is caught _not_ enforcing the norms. 

In [180]:
function update_payoff(landscape::Landscape, useMetaNorm=false)

    rows,cols = size(landscape)
    for i = 1:rows
        for j = 1:cols
            defect(landscape[i,j], landscape)
        end
    end        
    for i = 1:rows
        for j = 1:cols
            punish(landscape[i,j], landscape)
        end
    end
    if (useMetaNorm)
        for i = 1:rows
            for j = 1:cols
                metapunish(landscape[i,j], landscape)
            end
        end
    end
end

update_payoff (generic function with 2 methods)

In the first phase, each agent randomly generates its own likelihood of being caught, and decides whether or not to defect. If it does decide to defect, it gains some 'temptation' payoff and its neighbors lose some 'hurt' payoff. 

In [184]:
function defect(agent::Agent, landscape::Landscape, temptation=3, hurt=-1)
    
    agent.visibility = rand()
    if (agent.boldness < agent.visibility)
        agent.isBad = false
    else
        agent.isBad = true
        agent.payoff += temptation
        for neighbor = neighbors(agent, landscape)
            neighbor.payoff += hurt
        end
    end
end

defect (generic function with 3 methods)

This can be illustrated with a small test case. We create a 7x7 grid and constrain the agent in the top left-hand corner to always defect. When we run the defection code for just that agent, we see (on the left) that his own payoff has risen at the expense of his neighbors. 

In [185]:
test = initialize_landscape(7, 7)
agent = test[3,3]
agent.boldness = 1
defect(agent, test)

using Plots
heatmap(map((a)->a.payoff, test), title="Payoffs after agent [3,3] defects")


In the second phase, each agent looks at all its neighbors, and doles out punishment when the following conditions are met:

- The neighbor chose to defect
- The agent 'sees' the defection (with a probability given by `neighbor.visibility`)
- The agent chooses to punish (with a probability given by `agent.viciousness`)

The agent must explicitly track whether it chose mercy so that this can be incorporated in the metapunishment model.

In [59]:
function punish(agent::Agent, landscape::Landscape, penalty=-9, enforcement=-2)

    for neighbor = neighbors(agent, landscape)
                
        if (neighbor.isBad && neighbor.visibility >= rand()) 
            # Agent catches neighbor defecting

            if (agent.viciousness >= rand()) 
                # Agent chooses to punish
                neighbor.payoff += penalty
                agent.payoff += enforcement
                agent.isMerciful = false
                
            else 
                # Agent chooses mercy
                agent.isMerciful = true
            end            
        end
    end
end

punish (generic function with 3 methods)

This can be also be illustrated with a test case. Re-running this code snippet will reveal that the subset of neighbors who choose to punish is random. The punishers will themselves lose -2 payoff, but the offending agent will lose -9 each time. This choice of payoffs leads to a dramatically lose-lose situation (a negative-sum game?) -- because one agent chose to defect, every agent involved has lost utility.

In [61]:
test = initialize_landscape(7, 7)
agent = test[3,3]
agent.boldness = 1
defect(agent, test)
for neighbor = neighbors(agent, test)
    punish(neighbor, test)
end

heatmap(map((a)->a.payoff, test), title="Payoffs after neighbors punish agent [3,3]")


The final phase of `update_payoffs` is meta-punishment: The agents who chose not to punish (i.e. were not vicious enough) are themselves p

In [186]:
function metapunish(agent::Agent, landscape::Landscape, penalty=-3, enforcement=-1)

    for neighbor = neighbors(agent, landscape)
                
        if (neighbor.isMerciful &&              # Neighbor chose mercy
            neighbor.visibility >= rand() &&    # Agent detects merciful neighbor
            agent.viciousness >= rand())        # Agent chooses to punish
                    
            neighbor.payoff += penalty
            agent.payoff += enforcement
            
            # print("$(agent.location) is metapunishing $(neighbor.location)\n")
        end
    end
end

metapunish (generic function with 3 methods)

In [62]:
l = initialize_landscape(100, 75)
update_payoff(l)
p1 = heatmap(map((a)->a.payoff, l), title="Payoffs after iteration 1")
update_payoff(l)
p2 = heatmap(map((a)->a.payoff, l), title="Payoffs after iteration 2")
update_payoff(l)
p3 = heatmap(map((a)->a.payoff, l), title="Payoffs after iteration 3")
update_payoff(l)
p4 = heatmap(map((a)->a.payoff, l), title="Payoffs after iteration 4")

plot(p1,p2,p3,p4)

Now we introduce the evolutionary component. Once the expected payoffs are known, each agent may adjust their own strategy by examining the strategies and resultant payoffs of their neighbors. For now we use basic hill-climbing: Each agent takes a small step towards the strategy of the _best-performing_ neighbor.

In [173]:
function update_strategy(landscape::Landscape, stepsize=0.5, mutations=0.0)
    rows,cols = size(landscape)
    old_landscape = deepcopy(landscape)
    for i = 1:rows
        for j = 1:cols
            agent = landscape[i,j]
            
            if (mutations >= rand())
                # A fraction of agents choose their strategy randomly
                agent.boldness = rand()
                agent.viciousness = rand()
            else
                # Start with own strategy values
                b_best = agent.boldness
                v_best = agent.viciousness
                payoff_max = agent.payoff

                # Find best-performing neighbor
                for n = neighbors(agent, old_landscape)
                    if n.payoff > payoff_max
                        b_best = n.boldness
                        v_best = n.viciousness
                        payoff_max = n.payoff
                    end
                end

                # Update own strategy in direction of best-performing neighbor
                agent.boldness += stepsize * (b_best - agent.boldness)
                agent.viciousness += stepsize * (v_best - agent.viciousness)
                agent.boldness = max(min(agent.boldness, 1.0), 0.0)
                agent.viciousness = max(min(agent.viciousness, 1.0), 0.0)                
            end
            
            agent.payoff = 0.0
        end
    end  
end

update_strategy (generic function with 4 methods)

In [178]:
l = initialize_landscape(40, 30)
plots = []
for generation = 1:100
    for iteration = 1:50
        update_payoff(l)
    end
    p1 = heatmap(map((a)->a.boldness, l), title="Boldness")
    update_strategy(l,0.5,0.0)
end
plot(p1)


In [171]:
for agent = [l[1,1], l[6,18], l[17,26]]
    println("$(agent.location): B=$(agent.boldness), V=$(agent.viciousness) => payoff=$(agent.payoff)")
end

(1, 1): B=0.2993, V=0.701 => payoff=0.0
(6, 18): B=0.6426, V=0.1788 => payoff=0.0
(17, 26): B=0.5093, V=0.2163 => payoff=0.0
